In [ ]:
import os
rootdir = '/Users/matthewlarosa/Documents/code/ManueverID/Data/modifiedData/ObservedTrajectoryData/good_segmented_tsv'
newdir = '/Users/matthewlarosa/Documents/code/ManueverID/Data/modifiedData/ObservedTrajectoryData/good_animation_tsv'

def clean_bank(angle):
    global numBankInverted, prevBankAngle

    #Allows for 40degree movement per step
    if abs(angle - prevBankAngle) >= 320:
        #determine which direction
        # neg --> pos
        if angle > prevBankAngle:
            numBankInverted = numBankInverted - 1
        # pos --> neg
        else:
            numBankInverted = numBankInverted + 1

    prevBankAngle = angle
    if abs(numBankInverted) >= 1:
        angle = angle + (360*numBankInverted)

    return angle

def clean_heading(angle):
    global numHeadInverted, prevHeadAngle

    # Allows for 40degree movement per step
    #if this is violated, determine why
    if abs(angle - prevHeadAngle) >= 320:
        # determine which direction
        # neg --> pos
        if angle > prevHeadAngle:
            numHeadInverted = numHeadInverted - 1
        # pos --> neg
        else:
            numHeadInverted = numHeadInverted + 1

    prevHeadAngle = angle
    if abs(numHeadInverted) >= 1:
        angle = angle + (360 * numHeadInverted)

    return angle

#attempt to make the new directory for the animation sorties
if not (os.path.exists(newdir)):
    os.makedirs(newdir)

#fix the sorties and write them to file.
for subdir, dirs, files in os.walk(rootdir):
    for sortie in files:
        #open the file
        with open(subdir+"/"+sortie, "r") as bigFile:
            #Reads first line
            data = bigFile.readline().split('\t')

            # resetVars
            numBankInverted = 0
            numHeadInverted = 0
            prevBankAngle = float(data[10])
            prevHeadAngle = float(data[8])

            with open(newdir+"/"+ sortie, "w+") as smallFile:
                #write the first line of data that was already read
                smallFile.write('\t'.join(data))
                data = '\t'.join(data)
                #read next line until EOF
                while (data := bigFile.readline().split('\t')):
                    if not data == ['']:
                        #calculate new heading and bank angles
                        data[8] = str(clean_heading(float(data[8])))
                        data[10] = str(clean_bank(float(data[10]))) + "\n"

                        smallFile.write('\t'.join(data))
                    else:
                        break